# Detecting hot posts by classification

1. 目的：我們收到社群輿論資料共一萬多筆貼文（posts），想要從裡面建立一個分類模型來偵測什麼樣的貼文會「爆」（假設某則post有超過100則回文（comments），以PTT的語彙來說，就是爆了）。
2. 依變項（Dependent）：依變項為每則posts的回文數，把100則回文以上的文章視為1，把少於100則回文的視為0（切勿不可將回文數當成自變項，如果這麼做就會被扣很多分數）。但你也可以嘗試以不一樣的方式去切割，但要注意盡可能保持1/0兩個分類貼文數的平衡。
3. 自變項（Independent）：至少要以貼文內容作為自變項，但可以用貼文標題、貼文作者、貼文時間來訓練。除此之外，我們還提供了前10則回文，說不定影響一則貼文會不會爆的主要因素是前十則回文。你可以自己決定要不要把回文丟下去當自變項。本作業的要求只有要求要用貼文當自變項。
4. 參考版本：助教用simpletransformer寫了個[版本](https://colab.research.google.com/drive/1YQZNlzH_mo7_Q3XU8X2sIpkhrQZWVYar?usp=sharing)給各位參考，但作為示範助教僅使用了貼文內容作為特徵來訓練模型、也使用了較少的資料集（在時間上做了點切割）。你可以使用全部的貼文來預測，或者可以嘗試加入每則貼文的前10則回文來預測。


# 讀取資料
1. 貼文資料https://github.com/p4css/PSS/blob/master/data/ptt_post.pickle
2. 回文資料https://github.com/p4css/PSS/blob/master/data/ptt_comment10.pickle 
貼文與相對應的回文資料均以plink作為key，可以用pandas的merge將兩個資料集給連結起來做訓練。但要怎麼把前十則的回文化為貼文特徵，要想想。

In [5]:
import pandas as pd
df_post = pd.read_pickle('C:/Users/Sing-hao Ku/Desktop/Github/PSS/data/ptt_post.pickle')

df_post

,plink,board,pcontent,poster,ptitle,ptime,ipaddr,ip.len,Ncomment
0,https://www.ptt.cc/bbs/Gossiping/M.1119222660....,Gossiping,"\n\n 看到這推文,忍不住要寫一個飛機沒油的例子\n 很久之前在讀者文摘上看到,現憑印象寫...",Muroi (I Honestly Love You),Re: (問題)華航空難留言,Mon Jun 20 07:28:27 2005,138.130.212.179,1,944
1,https://www.ptt.cc/bbs/Gossiping/M.1119233779....,Gossiping,\n那不是為了反盜版\n那是為了捐錢給消基會\n當年美國唱了 WE ARE THE WORL...,JCC (JCC ),Re: 有沒有明天會更好的八卦,Mon Jun 20 10:21:53 2005,211.20.78.69,1,350
2,https://www.ptt.cc/bbs/Gossiping/M.1119257927....,Gossiping,\n\n我聽到的是：\n記者問張跟路有沒有要結婚\n 家人有沒有在催。。。巴拉巴拉的\n...,vancie (我不正 我很歪 ),Re: [新聞] 張震月的八卦有嗎？,Mon Jun 20 17:07:13 2005,※ 發信站: 批踢踢實業坊(ptt.cc),0,156
3,https://www.ptt.cc/bbs/Gossiping/M.1119258686....,Gossiping,"\n太想被M,得到P幣\n再八卦一下\n\n桌伯元先生真的不是台大法學碩士\n\n它是\n國...",J1 (andy),Re: [政商] 請問桌伯元的八卦?,Mon Jun 20 17:18:14 2005,220.141.159.23,1,160
4,https://www.ptt.cc/bbs/Gossiping/M.1119271499....,Gossiping,\n\n米其林是家輪胎公司\n米其林兄弟發行了一本只送不賣的輪胎手冊\n讓所有自動車 自行車...,ilcd (南特 巴黎 白朗峰!!! ),Re: 有沒有米其淋的美食還是什麼評鑑八卦,Mon Jun 20 20:53:24 2005,218.167.152.79,1,153
...,...,...,...,...,...,...,...,...,...
12144,https://www.ptt.cc/bbs/Gossiping/M.1594181669....,Gossiping,\n1.媒體來源:\nSETN三立新聞網\n\n2.記者署名:\n大陸中心／魏聖凌報導\n\...,SilverFlare (銀白閃焰),[新聞] 暴雨襲恐潰壩？湖北水庫爆變形垮塌當局嚇,Wed Jul 8 12:14:26 2020,36.238.77.37,1,60
12145,https://www.ptt.cc/bbs/Gossiping/M.1594183031....,Gossiping,\n1.媒體來源:\napple\n\n2.記者署名:\n黃任膺／台中報導\n\n3.完整新...,bigDwinsch (胸奴王),[新聞] 中市府副秘書長朱康震被爆劈4女 好巧！請假一周去健康檢查,Wed Jul 8 12:37:08 2020,110.26.72.88,1,57
12146,https://www.ptt.cc/bbs/Gossiping/M.1594183071....,Gossiping,\n\n現場已封鎖\n\n中國國家文物局截至7月7日下午16時，南方11省份有130餘處不可...,STAV72 (刁民黨黨務主委),[爆卦] 武當山五龍宮龍虎殿垮了！,Wed Jul 8 12:37:49 2020,49.216.167.110,1,189
12147,https://www.ptt.cc/bbs/Gossiping/M.1594190904....,Gossiping,"\n1.媒體來源:\n中時電子報,旺報\n\n2.記者署名:\n李侑珊\n\n3.完整新聞標...",Tcdamh (劣),[新聞] 陸配無法回台生子遭踢皮球 夫嘆：人道與,Wed Jul 8 14:48:22 2020,118.163.180.245,1,115


In [6]:
df_comment = pd.read_pickle('C:/Users/Sing-hao Ku/Desktop/Github/PSS/data/ptt_comment10.pickle')

df_comment

,tag,commentor,ccontent,ctimestamp,plink,cindex,ctotal
1491,推,hellgate,飛機沒有油了並不會就墜毀好嗎,140.119.162.51 06/20\n,https://www.ptt.cc/bbs/Gossiping/M.1119222660....,1,944
1492,推,nori,樓上的 請解釋,140.113.4.15 06/20\n,https://www.ptt.cc/bbs/Gossiping/M.1119222660....,2,944
1493,推,marrins,推讀者文摘,61.230.128.156 06/20\n,https://www.ptt.cc/bbs/Gossiping/M.1119222660....,3,944
1494,推,mmsg,後來有拍成電影...,220.134.50.133 06/20\n,https://www.ptt.cc/bbs/Gossiping/M.1119222660....,4,944
1495,推,Muroi,"能不能告訴我電影名稱,想找出來看 ^^",138.130.212.179 06/20\n,https://www.ptt.cc/bbs/Gossiping/M.1119222660....,5,944
...,...,...,...,...,...,...,...
32219866,推,solsol,民進黨又在管制言論自由了,07/08 15:32\n,https://www.ptt.cc/bbs/Gossiping/M.1594193428....,6,99
32219867,→,kaodio,鴿子本來很乖的 都是DPP帶壞我們家鴿子,07/08 15:32\n,https://www.ptt.cc/bbs/Gossiping/M.1594193428....,7,99
32219868,→,lingsk,他未經查證即刊載爭議內容，雖然不當,07/08 15:32\n,https://www.ptt.cc/bbs/Gossiping/M.1594193428....,8,99
32219869,→,solsol,馬英九時代就沒這樣,07/08 15:32\n,https://www.ptt.cc/bbs/Gossiping/M.1594193428....,9,99


# 切割資料集

In [ ]:
import numpy as np
import re

# 加上label，超過100則留言的為1
post['labels'] = pd.cut(post.comment_count, bins=[-np.inf, 100, np.inf], labels=[0, 1])
post.text = post.text.apply(lambda x: re.sub(r'\n+', '\n', x).strip())
post.text = post.text.apply(lambda x: x.replace('\n', ' ').strip()[:500])

post = post.sample(frac=0.5)

print(post.labels.value_counts())

# 訓練模型

# 預測結果
預測結果應包含`classification_report`並應用`from mlxtend.plotting import plot_confusion_matrix`繪製confision_matrix的視覺化結果。

# 分析預測結果
Q1. 你把什麼樣的特徵丟進去當自變項（最基本要有貼文，但你還可以丟貼文的發文者、時間、標題、前十則回文內容、回文作者）？
> ANSWER HERE

Q2. 你認為這個模型的分類能力好嗎？請說明模型訓練結果。
> ANSWER HERE

Q3. 你認為ptt貼文內容適合拿來做訓練資料嗎？為什麼？
> ANSWER HERE

Q4. 若加入其他參考特徵後，模型有什麼樣的改變呢（如果你沒做的話就不用回答）？
> ANSWER HERE

Q5. 如果你在Daily Scrum上要回報這個任務，請用100字以上、200字以內精簡且精準地回報模型訓練結果。
> ANSWER HERE